In [11]:
import os
import torch
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
from torch import optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

%matplotlib inline

In [12]:
DATASET_PATH = "./task/"
df = pd.read_csv(os.path.join(DATASET_PATH, "images_labelling.csv"), error_bad_lines=False) # neglect lin

In [13]:
problems = []
for idx, line in df.iterrows():
    if not os.path.exists(os.path.join(DATASET_PATH, 'images', str(line.boxid)+'.png')):
        print(idx)
        problems.append(idx)
df.drop(df.index[problems], inplace=True)

In [14]:
train_df, test_df = train_test_split(df, test_size=0.15, random_state=42)

In [15]:
train_df.head()

,boxid,label,class_
2957,50312,17,A20
2276,8389,9,B34
3630,393,3,B1
1960,15978,16,A19
2924,46882,17,A20


In [17]:
class PlayersDataset(Dataset):
    
    label_list = df['label'].unique()
    label2num = {label:i for i, label in enumerate(label_list)}
    num2label = {i:label for i, label in enumerate(label_list)}
    def __init__(self, root, dataframe, transform=None):
        super(PlayersDataset, self).__init__()
        self.dataframe = dataframe
        self.root = root
        if transform is None:
            transform = torchvision.transforms.Compose([
                torchvision.transforms.Resize((224, 224)),
                torchvision.transforms.ToTensor()
            ])
        self.transform = transform
        
    def __getitem__(self, idx):
        line = self.dataframe.iloc[idx]
        label = line.label
        label_id = self.label2num[label]
        img_path = os.path.join(self.root, str(line.boxid)+'.png')
        img = Image.open(img_path).convert('RGB')
        img_tensor = self.transform(img)
        return img_tensor, label_id
            
    def __len__(self):
        return len(self.dataframe)

In [18]:
# Construct dataset and dataloader
train_ds = PlayersDataset(os.path.join(DATASET_PATH, 'images'), train_df)
test_ds = PlayersDataset(os.path.join(DATASET_PATH, 'images'), test_df)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)

In [19]:
from torchvision.models import ResNet, resnet50
import torch.nn as nn

In [20]:
class PlayersModel(nn.Module):
  
    def __init__(self, num_classes):
        super(PlayersModel, self).__init__()
        backbone = resnet50(pretrained=True)
        backbone.fc = nn.Linear(backbone.fc.in_features, num_classes)
        self.backbone = backbone
    def forward(self, inp):
        return self.backbone(inp)

In [21]:
model = PlayersModel(len(df.label.unique()))

In [22]:
def train_val_loop(model, save_path):
    epochs = 50
    log_step = 20
    optimizer = optim.RMSprop(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model.to(device)
    loss_hist = []
    for epoch in range(1, epochs+1):
        # train phase
        model.train()
        for bth_num, batch in enumerate(train_loader, 1):
            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            loss = criterion(logits, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if bth_num % log_step == 0:
                print("\r Epoch: {}, # {} => Loss: {:.4f}".format(epoch, bth_num, loss.item()), flush=True, end='')
                loss_hist.append(loss.item())
        print()
        # Valid phase
        model.eval()
        total_preds = []
        total_labels = []
        for bth_num, batch in enumerate(test_loader, 1):
            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            preds = logits.argmax(dim=-1)
            total_labels.append(labels.cpu().data.numpy())
            total_preds.append(preds.cpu().data.numpy())
        total_preds = np.concatenate(total_preds)
        total_labels = np.concatenate(total_labels)
        acc = sum(total_preds == total_labels) / len(total_labels)
        print("Accuracy: {:.2%}".format(acc))
    
    # Save model weights
    torch.save(model.state_dict(), save_path)
    
    return loss_hist

In [23]:
loss_hist = train_val_loop(model, 'resnet50.pth')

 Epoch: 1, # 200 => Loss: 2.5175
Accuracy: 7.64%
 Epoch: 2, # 200 => Loss: 2.5812
Accuracy: 20.25%
 Epoch: 3, # 200 => Loss: 2.4764
Accuracy: 16.87%
 Epoch: 4, # 200 => Loss: 7.2754
Accuracy: 14.74%
 Epoch: 5, # 200 => Loss: 4.0742
Accuracy: 12.97%
 Epoch: 6, # 200 => Loss: 1.7070
Accuracy: 33.93%
 Epoch: 7, # 200 => Loss: 1.4631
Accuracy: 34.46%
 Epoch: 8, # 200 => Loss: 7.7219
Accuracy: 35.52%
 Epoch: 9, # 200 => Loss: 2.0489
Accuracy: 44.40%
 Epoch: 10, # 200 => Loss: 0.5173
Accuracy: 62.17%
 Epoch: 11, # 200 => Loss: 0.6944
Accuracy: 43.34%
 Epoch: 12, # 200 => Loss: 0.1125
Accuracy: 80.99%
 Epoch: 13, # 200 => Loss: 0.4371
Accuracy: 67.32%
 Epoch: 14, # 200 => Loss: 0.1763
Accuracy: 80.99%
 Epoch: 15, # 200 => Loss: 2.4998
Accuracy: 26.47%
 Epoch: 16, # 200 => Loss: 0.0446
Accuracy: 84.37%
 Epoch: 17, # 200 => Loss: 0.1265
Accuracy: 81.71%
 Epoch: 18, # 200 => Loss: 1.1125
Accuracy: 28.77%
 Epoch: 19, # 200 => Loss: 0.0091
Accuracy: 85.79%
 Epoch: 20, # 200 => Loss: 0.3135
Accurac